# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [2]:
nhl_df_copy = nhl_df.copy()
nhl_df_copy = nhl_df_copy[nhl_df_copy['year'] == 2018]
nhl_df_copy['team'] = nhl_df_copy['team'].str.lower()
nhl_df_copy['team'] = nhl_df_copy['team'].str.replace(r'\[.*\]', '', regex=True)
nhl_df_copy['team'] = nhl_df_copy['team'].str.replace(r'\*', '', regex=True)
nhl_df_copy['GP'] = pd.to_numeric(nhl_df_copy['GP'], errors='coerce')  # Convierte a numérico, convirtiendo no numéricos a NaN
nhl_df_copy = nhl_df_copy.dropna(subset=['GP'])  # Elimina las filas con valores NaN en la columna

In [3]:
cities_copy = cities.copy()
cities_copy['NHL'] = cities_copy['NHL'].str.lower()
cities_copy['Metropolitan area'] = cities_copy['Metropolitan area'].str.lower()
cities_copy['NHL'] = cities_copy['NHL'].str.replace(r'\[.*\]', '', regex=True)
cities_copy['NHL'] = cities_copy['NHL'].str.replace(r'\*', '', regex=True)
cities_copy['NHL'] = cities_copy['NHL'].replace("—", np.nan)
cities_copy['team'] = np.where(cities_copy['NHL'].notnull(), cities_copy['Metropolitan area'] + ' ' + cities_copy['NHL'], np.nan)

In [4]:
merged_df = pd.merge(nhl_df_copy, cities_copy, how='left', left_on='team', right_on='team')

In [5]:
not_in_merged_df = nhl_df_copy[~nhl_df_copy['team'].isin(merged_df['team'])]


In [6]:
unique_not_merged_team = not_in_merged_df['team'].unique()

print(unique_not_merged_team)
print(len(unique_not_merged_team))

[]
0


In [7]:
import pandas as pd

# Diccionario con equipos de la NHL
equipos_nhl = {
    'atlantic division': ['tampa bay lightning', 'florida panthers'],
    'metropolitan division': ['washington capitals', 'new jersey devils', 'carolina hurricanes', 'new york islanders', 'new york rangers'],
    'central division': ['minnesota wild', 'colorado avalanche', 'dallas stars'],
    'pacific division': ['vegas golden knights', 'anaheim ducks', 'san jose sharks', 'los angeles kings', 'arizona coyotes']
}

# Datos de población
population_data = {
    'tampa bay lightning': 3357096,
    'florida panthers': 679127,
    'washington capitals': 681170,
    'new jersey devils': 894446,
    'carolina hurricanes': 1130490,
    'new york islanders': 2418990,
    'new york rangers': 8537673,
    'minnesota wild': 3551036,
    'colorado avalanche': 4711142,
    'dallas stars': 7233323,
    'vegas golden knights': 632912,
    'anaheim ducks': 1334891,
    'san jose sharks': 1922200,
    'los angeles kings': 10137915,
    'arizona coyotes': 6392017
}

# Crear listas para construir el DataFrame
divisiones = []
equipos = []
poblacion = []

# Recorrer el diccionario para llenar las listas
for division, equipos_lista in equipos_nhl.items():
    for equipo in equipos_lista:
        divisiones.append(division)
        equipos.append(equipo)
        poblacion.append(population_data.get(equipo.lower(), None))  # Obtener la población según el nombre del equipo

# Crear el DataFrame
df_equipos_faltantes_nhl = pd.DataFrame({
    'Division': divisiones,
    'Equipo': equipos,
    'Population (2016 est.)[8]': poblacion
})


In [8]:
merged_df = pd.merge(merged_df, df_equipos_faltantes_nhl, how='left', left_on='team', right_on='Equipo')

In [9]:
merged_df['Metropolitan_area_consolited'] = merged_df['Metropolitan area'].fillna(merged_df['Division'])
merged_df['Population_consolited'] = merged_df['Population (2016 est.)[8]_x'].fillna(merged_df['Population (2016 est.)[8]_y'])
nuevo_merged_df = merged_df[['team', 'GP', 'W','L', 'OL', 'PTS','PTS%', 'GF', 'GA','SRS', 'SOS', 'RPt%','ROW', 'year', 'League','Metropolitan_area_consolited', 'Population_consolited']].copy()

In [10]:
columns_to_convert = ['W', 'L', 'OL', 'PTS', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'PTS%']
nuevo_merged_df[columns_to_convert] = nuevo_merged_df[columns_to_convert].astype(float)


In [11]:
nuevo_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   team                          31 non-null     object 
 1   GP                            31 non-null     float64
 2   W                             31 non-null     float64
 3   L                             31 non-null     float64
 4   OL                            31 non-null     float64
 5   PTS                           31 non-null     float64
 6   PTS%                          31 non-null     float64
 7   GF                            31 non-null     float64
 8   GA                            31 non-null     float64
 9   SRS                           31 non-null     float64
 10  SOS                           31 non-null     float64
 11  RPt%                          31 non-null     float64
 12  ROW                           31 non-null     float64
 13  year   

In [12]:
from scipy.stats import pearsonr

In [15]:
nuevo_merged_df['Win_Loss_Ratio'] = nuevo_merged_df['W'] / (nuevo_merged_df['W'] + nuevo_merged_df['L'])
population_by_region = nuevo_merged_df['Population_consolited']
win_loss_by_region = nuevo_merged_df['Win_Loss_Ratio']
correlation = nuevo_merged_df['Win_Loss_Ratio'].corr(nuevo_merged_df['Population_consolited'])

print(f"Correlation coefficient between Win/Loss Ratio and Population: {correlation}")

population_by_region = nuevo_merged_df['Population_consolited']
win_loss_by_region = nuevo_merged_df['Win_Loss_Ratio']

correlation_coefficient, p_value = pearsonr(population_by_region, win_loss_by_region)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")




Correlation coefficient between Win/Loss Ratio and Population: -0.1425271728944363
Pearson correlation coefficient: -0.14252717289443634
P-value: 0.4443522749327298


In [ ]:
from scipy.stats import pearsonr

# Assuming nuevo_merged_df is your DataFrame
# Assuming cities is your DataFrame containing metropolitan area information

# Extract relevant columns from nuevo_merged_df
relevant_columns = ['Metropolitan_area_consolited', 'Population_consolited', 'Win_Loss_Ratio']
subset_df = nuevo_merged_df[relevant_columns]

# Merge with cities DataFrame based on the 'Metropolitan_area_consolited' column
merged_df = pd.merge(subset_df, cities, how='inner', left_on='Metropolitan_area_consolited', right_on='Metropolitan area')

# Ensure 'Population_consolited' is in numeric format
merged_df['Population_consolited'] = pd.to_numeric(merged_df['Population_consolited'], errors='coerce')

# Extract the relevant columns for the correlation calculation
population_by_region = merged_df['Population_consolited']
win_loss_by_region = merged_df['Win_Loss_Ratio']

# Perform the correlation calculation
correlation_coefficient, p_value = pearsonr(population_by_region, win_loss_by_region)

print(f"Pearson correlation coefficient: {correlation_coefficient}")
print(f"P-value: {p_value}")


In [ ]:
print(len(population_by_region), len(win_loss_by_region))

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import pearsonr
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl_df_copy = nhl_df.copy()
    nhl_df_copy = nhl_df_copy[nhl_df_copy['year'] == 2018]
    nhl_df_copy['team'] = nhl_df_copy['team'].str.lower()
    nhl_df_copy['team'] = nhl_df_copy['team'].str.replace(r'\[.*\]', '', regex=True)
    nhl_df_copy['team'] = nhl_df_copy['team'].str.replace(r'\*', '', regex=True)
    nhl_df_copy['GP'] = pd.to_numeric(nhl_df_copy['GP'], errors='coerce')  # Convierte a numérico, convirtiendo no numéricos a NaN
    nhl_df_copy = nhl_df_copy.dropna(subset=['GP'])  # Elimina las filas con valores NaN en la columna

    cities_copy = cities.copy()
    cities_copy['NHL'] = cities_copy['NHL'].str.lower()
    cities_copy['Metropolitan area'] = cities_copy['Metropolitan area'].str.lower()
    cities_copy['NHL'] = cities_copy['NHL'].str.replace(r'\[.*\]', '', regex=True)
    cities_copy['NHL'] = cities_copy['NHL'].str.replace(r'\*', '', regex=True)
    cities_copy['NHL'] = cities_copy['NHL'].replace("—", np.nan)
    cities_copy['team'] = np.where(cities_copy['NHL'].notnull(), cities_copy['Metropolitan area'] + ' ' + cities_copy['NHL'], np.nan)

    merged_df = pd.merge(nhl_df_copy, cities_copy, how='left', left_on='team', right_on='team')
    equipos_nhl = {
    'atlantic division': ['tampa bay lightning', 'florida panthers'],
    'metropolitan division': ['washington capitals', 'new jersey devils', 'carolina hurricanes', 'new york islanders', 'new york rangers'],
    'central division': ['minnesota wild', 'colorado avalanche', 'dallas stars'],
    'pacific division': ['vegas golden knights', 'anaheim ducks', 'san jose sharks', 'los angeles kings', 'arizona coyotes']
    }

    # Datos de población
    population_data = {
        'tampa bay lightning': 3357096,
        'florida panthers': 679127,
        'washington capitals': 681170,
        'new jersey devils': 894446,
        'carolina hurricanes': 1130490,
        'new york islanders': 2418990,
        'new york rangers': 8537673,
        'minnesota wild': 3551036,
        'colorado avalanche': 4711142,
        'dallas stars': 7233323,
        'vegas golden knights': 632912,
        'anaheim ducks': 1334891,
        'san jose sharks': 1922200,
        'los angeles kings': 10137915,
        'arizona coyotes': 6392017
    }

    # Crear listas para construir el DataFrame
    divisiones = []
    equipos = []
    poblacion = []

    # Recorrer el diccionario para llenar las listas
    for division, equipos_lista in equipos_nhl.items():
        for equipo in equipos_lista:
            divisiones.append(division)
            equipos.append(equipo)
            poblacion.append(population_data.get(equipo.lower(), None))  # Obtener la población según el nombre del equipo

    # Crear el DataFrame
    df_equipos_faltantes_nhl = pd.DataFrame({
        'Division': divisiones,
        'Equipo': equipos,
        'Population (2016 est.)[8]': poblacion
    })

    merged_df = pd.merge(merged_df, df_equipos_faltantes_nhl, how='left', left_on='team', right_on='Equipo')  

    merged_df['Metropolitan_area_consolited'] = merged_df['Metropolitan area'].fillna(merged_df['Division'])
    merged_df['Population_consolited'] = merged_df['Population (2016 est.)[8]_x'].fillna(merged_df['Population (2016 est.)[8]_y'])
    nuevo_merged_df = merged_df[['team', 'GP', 'W','L', 'OL', 'PTS','PTS%', 'GF', 'GA','SRS', 'SOS', 'RPt%','ROW', 'year', 'League','Metropolitan_area_consolited', 'Population_consolited']].copy()  
    
    columns_to_convert = ['W', 'L', 'OL', 'PTS', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'PTS%']
    nuevo_merged_df[columns_to_convert] = nuevo_merged_df[columns_to_convert].astype(float)

    nuevo_merged_df['Win_Loss_Ratio'] = nuevo_merged_df['W'] / (nuevo_merged_df['W'] + nuevo_merged_df['L'])
    population_by_region = nuevo_merged_df['Population_consolited']
    win_loss_by_region = nuevo_merged_df['Win_Loss_Ratio']

    #correlation = nuevo_merged_df_2018['Win_Loss_Ratio'].corr(nuevo_merged_df_2018['Population_consolited'])
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [17]:
nhl_correlation()

AssertionError: Q1: There should be 28 teams being analysed for NHL

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values